# Today

  1. Geospatial Indexes and Queries in MongoDB
  2. Data Modeling Concepts

## Geospatial Indexes and Queries



In [4]:
%%bash
docker ps -a

CONTAINER ID        IMAGE                                                                                                                                COMMAND                  CREATED             STATUS                    PORTS                      NAMES
2e7905ecdfa7        mongo:latest                                                                                                                         "docker-entrypoint.s…"   2 days ago          Up 2 days                 0.0.0.0:27017->27017/tcp   dbms
34479006307d        436011858031.dkr.ecr.eu-west-1.amazonaws.com/ikkjo/watercolor:yakkety-FXAA-opencv3-pms-noise-scipy-sklearn-hg-wine-tinydb-requests   "/bin/bash"              3 months ago        Exited (1) 3 months ago                              silly_albattani


## Importing Coordinates of Resaurants and Neighbourhoods

From your last exercise you know how to import data into a MongoDB instance. Now, we are going to import JSON data instead of CSV files.

```bash
$ docker exec -it 2e79 bash
root@2e7905ecdfa7:/$ apt-get update
root@2e7905ecdfa7:/$ apt-get install -y wget
root@2e7905ecdfa7:/$ wget https://raw.githubusercontent.com/mongodb/docs-assets/geospatial/neighborhoods.json
root@2e7905ecdfa7:/$ wget https://raw.githubusercontent.com/mongodb/docs-assets/geospatial/restaurants.json
root@2e7905ecdfa7:/$ mongoimport ./restaurants.json -c restaurants
root@2e7905ecdfa7:/$ mongoimport ./neighborhoods.json -c neighborhoods
```

In [1]:
%%bash
docker exec 2e79 bash -c "apt-get update;apt-get install -y wget;wget https://raw.githubusercontent.com/mongodb/docs-assets/geospatial/neighborhoods.json;wget https://raw.githubusercontent.com/mongodb/docs-assets/geospatial/restaurants.json;mongoimport ./restaurants.json -c restaurants;mongoimport ./neighborhoods.json -c neighborhoods"

In [1]:
show collections

neighborhoods
restaurants

In [2]:
db.restaurants.find({name: "Morris Park Bake Shop"}) 

{ "_id" : ObjectId("55cba2476c522cafdb053add"), "location" : { "coordinates" : [ -73.856077, 40.848447 ], "type" : "Point" }, "name" : "Morris Park Bake Shop" }

That is, each imported reataurant has a point coordinate.

![](https://docs.mongodb.com/manual/_images/geospatial-single-point.png)

In [16]:
db.restaurants.find( { "location.coordinates": 
                       [-73.856077, 40.848447]})

{ "_id" : ObjectId("55cba2476c522cafdb053add"), "location" : { "coordinates" : [ -73.856077, 40.848447 ], "type" : "Point" }, "name" : "Morris Park Bake Shop" }

Each neighbourhood has a polygon defined by coordinates.

![](https://docs.mongodb.com/manual/_images/geospatial-polygon-hells-kitchen.png)

In [3]:
db.neighborhoods.findOne()

{
	"_id" : ObjectId("55cb9c666c522cafdb053a1c"),
	"geometry" : {
		"coordinates" : [
			[
				[
					-73.89138023380261,
					40.86170058826304
				],
				[
					-73.89106280613036,
					40.86152941211661
				],
				[
					-73.89106059904508,
					40.86147181520911
				],
				[
					-73.89115375546018,
					40.86091361616668
				],
				[
					-73.89174395026582,
					40.86017714781636
				],
				[
					-73.8922570762111,
					40.85940997630237
				],
				[
					-73.89299354227633,
					40.85838612046538
				],
				[
					-73.8944135507376,
					40.85643377580147
				],
				[
					-73.89477596003171,
					40.85592952563393
				],
				[
					-73.89549889768499,
					40.854749765891384
				],
				[
					-73.89560883346181,
					40.85457037009787
				],
				[
					-73.89644052566297,
					40.85490775915559
				],
				[
					-73.89709926839446,
					40.855172290942875
				],
				[
					-73.89831118626606,
					40.85406115105141
				],
				[
					-73.89863543744137,
					40.8539069754361
			

In [8]:
db.neighborhoods.find({name: "Midwood"})

{ "_id" : ObjectId("55cb9c666c522cafdb053a1b"), "geometry" : { "coordinates" : [ [ [ -73.94732672160579, 40.62916656720943 ], [ -73.94687439946738, 40.626773668128486 ], [ -73.94642266064318, 40.6243855672864 ], [ -73.94542442897408, 40.624495609598185 ], [ -73.94497031348948, 40.622104512937604 ], [ -73.94451833370498, 40.61971364709504 ], [ -73.94551663574785, 40.6196056119517 ], [ -73.94651373193828, 40.61949401914993 ], [ -73.9460669961243, 40.61710373918014 ], [ -73.94576359491893, 40.61550406422122 ], [ -73.94573691912028, 40.61536340675661 ], [ -73.94657366828137, 40.61478913113749 ], [ -73.94742608780807, 40.61422460055365 ], [ -73.94772682569696, 40.614169716182865 ], [ -73.94834517370118, 40.61413649663159 ], [ -73.94930527478495, 40.614030734516 ], [ -73.95026514882962, 40.613924351797365 ], [ -73.94999598103774, 40.612491461663225 ], [ -73.94996296695325, 40.61231572784935 ], [ -73.94993049319106, 40.612144650893164 ], [ -73.94978380450677, 40.61137175047764 ], [ -73.950705

## Finding a Restaurant by Location

In [4]:
db.restaurants.find().count()

25359

In [17]:
db.restaurants.find( { "location.coordinates": 
                       [-73.856077, 40.848447]
                     }).explain("executionStats")

{
	"queryPlanner" : {
		"plannerVersion" : 1,
		"namespace" : "test.restaurants",
		"indexFilterSet" : false,
		"parsedQuery" : {
			"location.coordinates" : {
				"$eq" : [
					-73.856077,
					40.848447
				]
			}
		},
		"winningPlan" : {
			"stage" : "COLLSCAN",
			"filter" : {
				"location.coordinates" : {
					"$eq" : [
						-73.856077,
						40.848447
					]
				}
			},
			"direction" : "forward"
		},
		"rejectedPlans" : [ ]
	},
	"executionStats" : {
		"executionSuccess" : true,
		"nReturned" : 1,
		"executionTimeMillis" : 12,
		"totalKeysExamined" : 0,
		"totalDocsExamined" : 25359,
		"executionStages" : {
			"stage" : "COLLSCAN",
			"filter" : {
				"location.coordinates" : {
					"$eq" : [
						-73.856077,
						40.848447
					]
				}
			},
			"nReturned" : 1,
			"executionTimeMillisEstimate" : 0,
			"works" : 25361,
			"advanced" : 1,
			"needTime" : 25359,
			"needYield" : 0,
			"saveState" : 198,
			"restoreState" : 198,
			"isEOF" : 1,
			"invalidates" : 0,
			"directi

## More than just finding a precise location


  * Find a matching polygon from point with `$geoIntersects`
  * Find all points in a ploygon with `$geoWithin`
  * Find all point in viccinity of another point with `$nearSphere`
  
All of these operations require a geospatial index on our dataset.

MongoDB supports two types of spatial indexes one on spheres `2dsphere` and one one planes `2d`.

## Creating the Index

To make this work, the geometry data in the location field must follow the GeoJSON (https://docs.mongodb.com/manual/reference/geojson/) format.

#### Point

```json
{ type: "Point", coordinates: [ 40, 5 ] }
```

#### Polygon

```json
{
  type: "Polygon",
  coordinates: [ [ [ 0 , 0 ] , [ 3 , 6 ] , [ 6 , 1 ] , [ 0 , 0  ] ] ]
}```

In [19]:
db.restaurants.createIndex({ location: "2dsphere" })

{
	"createdCollectionAutomatically" : false,
	"numIndexesBefore" : 1,
	"numIndexesAfter" : 2,
	"ok" : 1
}

### A spherical index, why???

Because it makes your live as programmer easier...

Do you want to compute something like this for all of your queries?

![](https://docs.mongodb.com/manual/_images/geospatial-spherical-square.png)

### 1. Find the a Neighborhood for Location with `$geoIntersects`

In [23]:
db.neighborhoods.findOne({ geometry: { 
                             $geoIntersects: { 
                               $geometry: { 
                                 type: "Point", 
                                 coordinates: [ -73.93414657, 40.82302903 ] 
                               } 
                             } 
                            } 
                          })

{ "_id" : ObjectId("55cb9c666c522cafdb053a68"), "geometry" : { "coordinates" : [ [ [ -73.93383000695911, 40.81949109558767 ], [ -73.93411701695138, 40.81955053491088 ], [ -73.93431276819767, 40.81962986684897 ], [ -73.93440040009484, 40.819667782434465 ], [ -73.93439036753757, 40.81958743539698 ], [ -73.93438112852347, 40.81932807562949 ], [ -73.9342948780807, 40.81890498677685 ], [ -73.93423576072603, 40.818649319251364 ], [ -73.93408482013453, 40.81793086150489 ], [ -73.9340746397773, 40.81782651394852 ], [ -73.93412816761214, 40.8177339625139 ], [ -73.93423519549512, 40.81755488121525 ], [ -73.93445400138874, 40.81718852906137 ], [ -73.93463790272767, 40.81690804797262 ], [ -73.93487343834981, 40.8165278475176 ], [ -73.93575548210782, 40.81532558429708 ], [ -73.93613554330996, 40.8147881266393 ], [ -73.93620957748075, 40.81467385042275 ], [ -73.93668854882762, 40.81404163072326 ], [ -73.93708163694987, 40.81350614105259 ], [ -73.93714398424214, 40.81342104144455 ], [ -73.93761714951

### 2. Find all Points in a Polygon with `$geoWithin`

In [6]:
var whereAmI = { geometry: 
                   { $geoIntersects: 
                     { $geometry: 
                       { type: "Point", 
                         coordinates: [ -73.93414657, 40.82302903 ] 
                       } 
                     } 
                   } 
               };
var neighborhood = db.neighborhoods.findOne( whereAmI );
var doc2 = { location: 
             { $geoWithin: 
               { $geometry: neighborhood.geometry } 
             } 
           };
db.restaurants.find( doc2 )

{ "_id" : ObjectId("55cba2476c522cafdb057f93"), "location" : { "coordinates" : [ -73.93781899999999, 40.8073089 ], "type" : "Point" }, "name" : "Perfect Taste" }
{ "_id" : ObjectId("55cba2476c522cafdb058b1a"), "location" : { "coordinates" : [ -73.943046, 40.807862 ], "type" : "Point" }, "name" : "Event Productions Catering & Food Services" }
{ "_id" : ObjectId("55cba2476c522cafdb054824"), "location" : { "coordinates" : [ -73.9446889, 40.8087276 ], "type" : "Point" }, "name" : "Sylvia'S Restaurant" }
{ "_id" : ObjectId("55cba2476c522cafdb0576a9"), "location" : { "coordinates" : [ -73.9450154, 40.808573 ], "type" : "Point" }, "name" : "Corner Social" }
{ "_id" : ObjectId("55cba2476c522cafdb0576c1"), "location" : { "coordinates" : [ -73.9449154, 40.8086991 ], "type" : "Point" }, "name" : "Cove Lounge" }
{ "_id" : ObjectId("55cba2476c522cafdb056a56"), "location" : { "coordinates" : [ -73.9508604, 40.8111432 ], "type" : "Point" }, "name" : "Manna'S Restaurant" }
{ "_id" : ObjectId("55cba247

![](https://docs.mongodb.com/manual/_images/geospatial-all-restaurants.png) 

### 3. Find Points in Viccinity with `$geoWithin` and `$nearSphere`


The following query shows all restaurants within 500m distance to the given location.

See more for convertion of units https://docs.mongodb.com/manual/tutorial/calculate-distances-using-spherical-geometry-with-2d-geospatial-indexes/.

Keep in mind that the following query returns the matching locations unordered.

In [10]:
db.restaurants.find({ location:
                      { $geoWithin:
                        { $centerSphere: [ 
                          [ -73.93414657, 40.82302903 ], 
                          500 / 6378100.0 ] } 
                        } 
                    })

{ "_id" : ObjectId("55cba2476c522cafdb056b6a"), "location" : { "coordinates" : [ -73.93011659999999, 40.8219403 ], "type" : "Point" }, "name" : "Applebee'S Neighborhood Grill & Bar" }
{ "_id" : ObjectId("55cba2476c522cafdb0578b3"), "location" : { "coordinates" : [ -73.93011659999999, 40.8219403 ], "type" : "Point" }, "name" : "Marisco Centro Seafood Restaurant  & Bar" }
{ "_id" : ObjectId("55cba2476c522cafdb058c83"), "location" : { "coordinates" : [ -73.9316894, 40.8231974 ], "type" : "Point" }, "name" : "Gotham Stadium Tennis Center Cafe" }
{ "_id" : ObjectId("55cba2476c522cafdb058c63"), "location" : { "coordinates" : [ -73.9303724, 40.8234978 ], "type" : "Point" }, "name" : "Chuck E. Cheese'S" }
{ "_id" : ObjectId("55cba2476c522cafdb056b96"), "location" : { "coordinates" : [ -73.9308109, 40.82594580000001 ], "type" : "Point" }, "name" : "Dunkin Donuts (Inside Gulf Gas Station On North Side Of Maj. Deegan Exwy- After Exit 13 - 233 St.)" }
{ "_id" : ObjectId("55cba2476c522cafdb05814c")

The `$nearSphere` operator returns the matching locations in sorted order from nearest to farthest.

`$maxDistance` and `$minDistance` are specified in meters.

In [7]:
db.restaurants.find({ location: 
                      { $nearSphere: 
                        { $geometry: 
                          { type: "Point", 
                            coordinates: [ -73.93414657, 40.82302903 ] 
                          }, 
                        $maxDistance: 500 } } })

{ "_id" : ObjectId("55cba2476c522cafdb058c83"), "location" : { "coordinates" : [ -73.9316894, 40.8231974 ], "type" : "Point" }, "name" : "Gotham Stadium Tennis Center Cafe" }
{ "_id" : ObjectId("55cba2476c522cafdb05864b"), "location" : { "coordinates" : [ -73.9378967, 40.823448 ], "type" : "Point" }, "name" : "Tia Melli'S Latin Kitchen" }
{ "_id" : ObjectId("55cba2476c522cafdb058c63"), "location" : { "coordinates" : [ -73.9303724, 40.8234978 ], "type" : "Point" }, "name" : "Chuck E. Cheese'S" }
{ "_id" : ObjectId("55cba2476c522cafdb0550aa"), "location" : { "coordinates" : [ -73.93795159999999, 40.823376 ], "type" : "Point" }, "name" : "Domino'S Pizza" }
{ "_id" : ObjectId("55cba2476c522cafdb0548e0"), "location" : { "coordinates" : [ -73.9381738, 40.8224212 ], "type" : "Point" }, "name" : "Red Star Chinese Restaurant" }
{ "_id" : ObjectId("55cba2476c522cafdb0578b3"), "location" : { "coordinates" : [ -73.93011659999999, 40.8219403 ], "type" : "Point" }, "name" : "Marisco Centro Seafood R

## References

  * https://docs.mongodb.com/manual/tutorial/geospatial-tutorial/
  

# Your Turn!


![](http://www.twenty19.com/blog/wp-content/uploads/2017/07/typing2.gif)


Try it on your own. Import the cities file that you need for your project, e.g.,

  * http://download.geonames.org/export/dump/cities15000.zip   
  * http://download.geonames.org/export/dump/cities5000.zip 
  * http://download.geonames.org/export/dump/cities1000.zip 

and create a geospatial index, so that you can find all cities, which are in 50 kilometers distance of Copenhagen `(55.67594, 12.56553)` for example.

# Data Modeling Concepts

  > Database schema design is the process of choosing the best representation for a data set, given the features of the database system, the nature of the data, and the application requirements.
  >
  > _MongoDB in Action_

  > The key challenge in data modeling is balancing the needs of the application, the performance characteristics of the database engine, and the data retrieval patterns. When designing data models, always consider the application usage of the data (i.e. queries, updates, and processing of the data) as well as the inherent structure of the data itself.
  >
  > https://docs.mongodb.com/manual/core/data-modeling-introduction/
  


## Embedded Data Models - Embedded Documents

![](https://docs.mongodb.com/manual/_images/data-model-denormalized.bakedsvg.svg)

Use embedded data models for:

  * modeling _contains_ relationships between entities.
  * modeling _one-to-many_ relationships between entities.

## Normalized Data Models - References

![](https://docs.mongodb.com/manual/_images/data-model-normalized.bakedsvg.svg)

Use normalized data models:

  * when _embedding_ would result in duplication of data but would not provide sufficient read performance advantages to outweigh the implications of the duplication.
  * to model large _hierarchical data sets_

https://docs.mongodb.com/manual/core/data-model-design/

### What to consider under data modelling

  * Atomicity of Write Operations
  * Document Growth
  * Data Use and Performance
  * Document Growth 
  * Indexes
  * Collection Contains Large Number of Small Documents
  * Data Lifecycle Management
  * Sharding

https://docs.mongodb.com/manual/core/data-model-operations/

## Sharding?

![](https://docs.mongodb.com/manual/_images/sharding-range-based.bakedsvg.svg)

## Data Model Examples and Patterns

### Model Relationships Between Documents

#### One-to-One Relationships with Embedded Documents

```json
{
   _id: "joe",
   name: "Joe Bookreader"
}

{
   patron_id: "joe",
   street: "123 Fake Street",
   city: "Faketon",
   state: "MA",
   zip: "12345"
}
```


```json
{
   _id: "joe",
   name: "Joe Bookreader",
   address: {
              street: "123 Fake Street",
              city: "Faketon",
              state: "MA",
              zip: "12345"
            }
}
```

#### One-to-Many Relationships with Embedded Documents

```json
{
   _id: "joe",
   name: "Joe Bookreader"
}
```

```json
{
   patron_id: "joe",
   street: "123 Fake Street",
   city: "Faketon",
   state: "MA",
   zip: "12345"
}
```

```json
{
   patron_id: "joe",
   street: "1 Some Other Street",
   city: "Boston",
   state: "MA",
   zip: "12345"
}
```

#### One-to-Many Relationships with Embedded Documents II


```json
{
   _id: "joe",
   name: "Joe Bookreader",
   addresses: [
                {
                  street: "123 Fake Street",
                  city: "Faketon",
                  state: "MA",
                  zip: "12345"
                },
                {
                  street: "1 Some Other Street",
                  city: "Boston",
                  state: "MA",
                  zip: "12345"
                }
              ]
 }
 ```

#### One-to-Many Relationships with Document References

```json
{
   title: "MongoDB: The Definitive Guide",
   author: [ "Kristina Chodorow", "Mike Dirolf" ],
   published_date: ISODate("2010-09-24"),
   pages: 216,
   language: "English",
   publisher: {
              name: "O'Reilly Media",
              founded: 1980,
              location: "CA"
            }
}
```

```json
{
   title: "50 Tips and Tricks for MongoDB Developer",
   author: "Kristina Chodorow",
   published_date: ISODate("2011-05-06"),
   pages: 68,
   language: "English",
   publisher: {
              name: "O'Reilly Media",
              founded: 1980,
              location: "CA"
            }
}
```

#### One-to-Many Relationships with Document References II

```json
{
   name: "O'Reilly Media",
   founded: 1980,
   location: "CA",
   books: [123456789, 234567890, ...]
}

{
    _id: 123456789,
    title: "MongoDB: The Definitive Guide",
    author: [ "Kristina Chodorow", "Mike Dirolf" ],
    published_date: ISODate("2010-09-24"),
    pages: 216,
    language: "English"
}

{
   _id: 234567890,
   title: "50 Tips and Tricks for MongoDB Developer",
   author: "Kristina Chodorow",
   published_date: ISODate("2011-05-06"),
   pages: 68,
   language: "English"
}
```

#### One-to-Many Relationships with Document References III


```json
{
   _id: "oreilly",
   name: "O'Reilly Media",
   founded: 1980,
   location: "CA"
}
```

```json
{
   _id: 123456789,
   title: "MongoDB: The Definitive Guide",
   author: [ "Kristina Chodorow", "Mike Dirolf" ],
   published_date: ISODate("2010-09-24"),
   pages: 216,
   language: "English",
   publisher_id: "oreilly"
}
```

```json
{
   _id: 234567890,
   title: "50 Tips and Tricks for MongoDB Developer",
   author: "Kristina Chodorow",
   published_date: ISODate("2011-05-06"),
   pages: 68,
   language: "English",
   publisher_id: "oreilly"
}
```

### Modeling Tree Structures with Parent References

![](https://docs.mongodb.com/manual/_images/data-model-tree.bakedsvg.svg)


```javascript
db.categories.insert( { _id: "MongoDB", parent: "Databases" } )
db.categories.insert( { _id: "dbm", parent: "Databases" } )
db.categories.insert( { _id: "Databases", parent: "Programming" } )
db.categories.insert( { _id: "Languages", parent: "Programming" } )
db.categories.insert( { _id: "Programming", parent: "Books" } )
db.categories.insert( { _id: "Books", parent: null } )
```

#### Getting the Parent

```javascript
db.categories.findOne( { _id: "MongoDB" } ).parent
```

#### Getting Children

```javascript
db.categories.find( { parent: "Databases" } )
```


#### Modeling Tree Structures with Child References

![](https://docs.mongodb.com/manual/_images/data-model-tree.bakedsvg.svg)


```javascript
db.categories.insert( { _id: "MongoDB", children: [] } )
db.categories.insert( { _id: "dbm", children: [] } )
db.categories.insert( { _id: "Databases", children: [ "MongoDB", "dbm" ] } )
db.categories.insert( { _id: "Languages", children: [] } )
db.categories.insert( { _id: "Programming", children: [ "Databases", "Languages" ] })
db.categories.insert( { _id: "Books", children: [ "Programming" ] } )
```

#### Getting the Children

```javascript
db.categories.findOne( { _id: "Databases" } ).children
```

#### Getting Parent and Siblings

```javascript
db.categories.find( { children: "MongoDB" } )
```



# Your Turn!

![](http://www.twenty19.com/blog/wp-content/uploads/2017/07/typing2.gif)

Model the tree above in your MongoDB using the following three strategies:

  * Arrays of Ancestors, see https://docs.mongodb.com/manual/tutorial/model-tree-structures-with-ancestors-array/
  * Materialized paths, see https://docs.mongodb.com/manual/tutorial/model-tree-structures-with-materialized-paths/
  * Nested sets, https://docs.mongodb.com/manual/tutorial/model-tree-structures-with-nested-sets/

Run the corresponding queries and describe when to use which strategy.


Since it was asked during the last session: How do I enforce certain schema constraints? The following document provides information on how to do that in the DB engine instead of application code, see https://docs.mongodb.com/manual/core/document-validation/